# 01 Webscraping

### TODO (Nice to have)

- input = cphbusiness.dk OK
- lav input til en string istedet for liste IKKE RELEVANT
- gem scrape med side navnet + pickle dump til side navn OK
- QOL kun lav sleep når man henter mere end 5 pages. OK
- QQL omskrive metode navne
- skriv en funktion som renser review data/replace & tilføj lowercase clean fra datacleaning notebook OK
- Stor QQL lav en celle til sidst som kører det hele.

In [2]:
import bs4
import csv
import requests
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import random
import pickle
import time

workingProxyList=[] # used for proxies (free ones are too slow)
allShopsRequest = []
totalpageprShop=[] # used in multiThreadGetTotalPages
allPagesRequest = [] # used in multiThreadDownloader
reviewErrorCount = [] # used in multithreadwritecsvfile
dataDir = './data/'

In [ ]:
#Kør kun denne celle hvis du vil bruge proxy

#with open('ipList2', 'rb') as file:
#    workingProxyList = pickle.load(file)

In [ ]:
#Websites du vil scrape fra på trustpilot.dk
allShopsRequest.append("ny-emalje.dk")
allShopsRequest.append("iwao.dk")
print("OK")

In [3]:
#Load tidligere gemte data i et rå/usorteret format for at undgå at skulle scrape igen.
def loadRawData(*args):
    allPagesRequest = []
    for arg in args:
        with open(dataDir+arg,'rb') as file:
            allPagesRequest = allPagesRequest+ pickle.load(file)
    return allPagesRequest

    
    
allPagesRequest = loadRawData('ny-emalje','iwao')
# for i in allPagesRequest:
#     print(i.url)

In [1]:
#Frasortere emojis
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

#Frasortere linjeskift, comma, og dobbelt mellemrum & gør al tekst lowercase.
def data_cleaning(data):
    data = data.replace(",", " ")
    data = data.replace(' "" ', "")
    data = data.replace('  ', "")
    data = data.replace(' \n ', "")
    data = data.lower()
    return data


#Splitter hver url i listen, og returnere hjemmesidenavnet.
def createFiles(url):
    filename = str(url.url)
    filename = filename.split('.')

    if len(filename) == 4:
        filename = filename[2].split('/')
        filename = filename[2]
    else:
        filename = filename[3]
    return filename


#Tjekker om filen eksistere, så den ikke bliver appended til igen. 
def checkFileExists(filename):
    
    try:
        with open(filename,'rb') as file:
            return 0
    except:
        return 1


print("OK")

OK


In [ ]:
#Henter antal review sider for hver butik i listen og gemmer det som en liste af tuples.
def multiThreadGetTotalPages(shop):
    shop = 'https://dk.trustpilot.com/review/'+shop
    loadingbar.update(1)
    listofpages=""
    totalpages=None
    req = requests.get(shop, timeout=10)
    soup = bs4.BeautifulSoup(req.content, 'html.parser')

    try:
        totalpages = soup.find('a', attrs={'name':'pagination-button-last'}).text
        shopAndPage = {'shop': shop,'pages': totalpages}
        totalpageprShop.append(shopAndPage)
    except: 
        listofpages = [7,6,5,4,3,2,1]
        for content in listofpages:
            try:
                totalpages = soup.find('a', attrs={'name':'pagination-button-'+str(content)}).text
                if totalpages is not None:
                    shopAndPage = {'shop': shop,'pages': totalpages}
                    totalpageprShop.append(shopAndPage)
                    break
                
            except Exception as e:
                pass
           
    if len(totalpageprShop) == 0:
        shopAndPage = {'shop': shop,'pages': 1}
        totalpageprShop.append(shopAndPage)
 

with tqdm(total=len(allShopsRequest)) as loadingbar: 
    with ThreadPoolExecutor(2) as ex:
        ex.map(multiThreadGetTotalPages, allShopsRequest)
        
print('Ok')

In [ ]:
#Henter alle siderne fra trustpilot og tilføjet dem i en liste. Før noget filtering.
def multiThreadDownloader(shop,page):
    loadingbar.update(1)
    page = page+1
    url = shop+'?page='+str(page)
    
    #Overvej at droppe proxy.. Servers er langsomme, timer ofte ud eller virker slet ikke. 
    #Selvom de er testet 5min forinden.
    if len(workingProxyList) > 0:
        for ip in workingProxyList:
            try:
                randomIpNumber = random.randint(0,len(workingProxyList))
                allPagesRequest.append(requests.get(url, timeout=15, proxies=workingProxyList[randomIpNumber]))
                break
            except:
                pass
    else:
        allPagesRequest.append(requests.get(url, timeout=10))
    
    if allpages > 5:
        time.sleep(10)


allpages=0    
for x in totalpageprShop:
    allpages = allpages+int(x['pages'])


with tqdm(total=allpages) as loadingbar:
    for i in totalpageprShop:
        digitlist = []
        res = int(i['pages'])
        for digit in range(res):
            digitlist.append(digit)
        with ThreadPoolExecutor(1) as ex:
            ex.map(multiThreadDownloader,[i['shop']] * len(digitlist), digitlist)
            

            


In [ ]:
#Gemmer listen som indeholder alle requests data i rå format. Før filtering. 
def rawDataDump(allPagesRequest):
    dumpList = []
    tmp=""
    for x in allPagesRequest:
        dumpname = createFiles(x)
        if dumpname in x.url:
            if tmp == "":
                dumpList.append(x)
                tmp=dumpname
                with open(dataDir+dumpname, 'wb') as output_file:
                    pickle.dump(dumpList,output_file)
            elif tmp==dumpname:
                dumpList.append(x)
                with open(dataDir+dumpname, 'wb') as output_file:
                    pickle.dump(dumpList,output_file)   
            else:
                dumpList = []
                tmp=""
                dumpList.append(x)
                with open(dataDir+dumpname, 'wb') as output_file:
                    pickle.dump(dumpList,output_file)


            
rawDataDump(allPagesRequest)

In [1]:
#Filtere den gå data med bs4 og skriver til en csv med det navn som request url'et indholder.
fileAlreadyExists = 0  
def multithreadwritecsvfile(page):
    loadingbar.update(1)
    
    filename = dataDir+createFiles(page)+'.csv'
    
    fileAlreadyExists = checkFileExists(filename)
    if fileAlreadyExists == 1:    
        with open(filename, 'a', newline='', encoding='utf-8') as output_file:
            output_writer = csv.writer(output_file)

            if output_file is None:
                output_writer.writerow(['rating','review'])

            soup = bs4.BeautifulSoup(page.content,'html.parser')
            for content in soup.find_all('section', attrs={'class':'styles_reviewContentwrapper__zH_9M'}):
                rating=None
                splitted=None
                review=None

                try: 
                    rating = content.find('img',alt = True)
                    splitted = rating.get('alt').split()
                except Exception as e:
                    print('No rating found. Skipping...', e)

                try: 
                    review = content.find('p', attrs={'class':'typography_typography__QgicV typography_body__9UBeQ typography_color-black__5LYEn typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3'}).text
                    if len(review) > 2:
                        review = remove_emojis(review)
                        review = data_cleaning(review)
                        output_writer.writerow(['__label__'+splitted[2]+" ", review])
                except Exception as e:
                    reviewErrorCount.append(e);            

  
with tqdm(total=len(allPagesRequest)) as loadingbar: 
    with ThreadPoolExecutor(4) as ex:
        
        ex.map(multithreadwritecsvfile, allPagesRequest)

print('Done. Skipped reviews:', len(reviewErrorCount))



NameError: name 'tqdm' is not defined